In [ ]:
%load_ext autoreload
%autoreload 2
from matplotlib import pyplot as plt

from factortest.preprocess_data.load_data import load_sp500_tickers, fetch_data_from_yahoo, fetch_fama_french_monthly_factors, generate_monthly_returns_from_daily_price_data
from factortest.estimate.factor_model import estimate_ff3, estimate_capm, estimate_factor_exposure

import datetime

import warnings

warnings.filterwarnings('ignore')


In [ ]:
companies = load_sp500_tickers()
companies

In [ ]:
start = datetime.datetime(2000, 1, 1)
end = datetime.datetime(2023, 1, 1)
market_data = fetch_data_from_yahoo(companies, start,end)
market_data

In [ ]:
factors = fetch_fama_french_monthly_factors()
factors

In [ ]:
price_column = 'Adj Close'
ticker_column = 'Symbol'
date_column = 'Date'
monthly_returns = generate_monthly_returns_from_daily_price_data(market_data, price_column, ticker_column, date_column)
monthly_returns

In [ ]:
monthly_returns = monthly_returns.join(factors, on='Date', how='left')
monthly_returns['Excess Return'] = monthly_returns['Monthly Returns'] - monthly_returns['RF']

In [ ]:
monthly_returns.sort_values(by='Date')
monthly_returns['ex_ret_1'] = monthly_returns.groupby('Symbol')['Excess Return'].shift(-1)
monthly_returns = monthly_returns.dropna(subset=['ex_ret_1']).dropna(subset=['HML'])
monthly_returns.describe()

In [ ]:
LHS = 'ex_ret_1'
MARKET = 'Mkt-RF'
FF = ['Mkt-RF', 'SMB', 'HML']

In [ ]:
factor_betas, factor_pvalues = estimate_ff3(monthly_returns, left_hand_side_variable=LHS, right_hand_side_variables=FF,
                                      ticker_column=ticker_column)
factor_betas

In [ ]:
capm_betas, capm_pvalues = estimate_capm(monthly_returns, left_hand_side_variable=LHS,
                                           right_hand_side_variable=MARKET,
                                           ticker_column=ticker_column)
capm_betas

In [ ]:
ticker = 'TSLA'

to_plot = monthly_returns[monthly_returns['Symbol'].isin([ticker])]
to_plot.plot(kind='scatter', x='Mkt-RF', y='Excess Return', figsize=(8, 8))
plt.plot(to_plot['Mkt-RF'],
         capm_betas['Mkt-RF'].filter(like=ticker).values.astype(float) * to_plot['Mkt-RF'] + capm_betas[
             'const'].filter(like=ticker).values.astype(float), '-', color='r')
plt.text(-0.15, -1.3,
         f"P-value of market beta: {capm_pvalues['Mkt-RF'].filter(like=ticker).values.item(0):.3f}")
plt.show()

In [ ]:
monthly_returns_2 = monthly_returns.join(factor_betas, on='Symbol', how='left', rsuffix='_b')
monthly_returns_2

In [ ]:
factor_beta_column_names = ['Mkt-RF_b', 'SMB_b', 'HML_b']
capm_beta_column_name = ['Mkt-RF_b']
LHS = 'ex_ret_1'

lambda_pd, lambda_pvalues = estimate_factor_exposure(monthly_returns_2, expected_return=LHS, factor_betas=factor_beta_column_names)
lambda_pd

In [ ]:
lambda_pd.mean().rename('Mean factor exposure')

In [ ]:
window_length = 12

ax1 = plt.subplot2grid((1, 3), (0, 0))
ax2 = plt.subplot2grid((1, 3), (0, 1), colspan=2)
ax2.margins(0.01)
lambda_pd.mean().plot.barh(ax=ax1)
lambdas0 = lambda_pd.rolling(window_length).mean().dropna()
lambdas0.plot(lw=2, figsize=(17, 8), ax=ax2)
ax2.legend(bbox_to_anchor=(1.025, 1.05))
plt.show()

lambda_pd.rolling(window_length).mean().dropna().plot(lw=2, figsize=(14, 20), subplots=True, sharey=True, sharex=True)
plt.show()